In [3]:
#1 Loading the Data
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
from datetime import datetime, timedelta
if os.path.isdir('Graphs') == False:
    os.mkdir('Graphs')
burglar_df = pd.read_csv('Burglars.csv')

In [4]:
#2 Grouping by LSOA
groups = burglar_df.groupby('LSOA name')
grouped_df = {name: df for name, df in groups}

In [5]:
#3 Define month-grabbing function:
def get_per_month_count(unf_df, fil_df):
    months = unf_df['Month'].unique()
    per_month = {}
    for i in months:
        per_month[i] = len(fil_df[fil_df['Month'] == i])
    return per_month

In [6]:
#4 Generating the per-month-year bar charts and dictionaries:
LSOA_per_month = {}
for i in grouped_df.keys():
    df = grouped_df[i]
    groups = df.groupby('Month')
    month_df = get_per_month_count(burglar_df, df)
    LSOA_per_month[i] = month_df
    months = list(month_df.keys())
    step = max(1, len(months) // 15)


In [7]:
#5 Automatic Recency problem analysis: Define Capturing of Points and Gaps
print(LSOA_per_month.keys())
point_gap = {}
for i in LSOA_per_month.keys():
    df = LSOA_per_month[i]
    gap = []
    point = []
    neither = []
    for month in df.keys():
        if df[month] == 0:
            gap.append(month)
        elif df[month] == 1:
            point.append(month)
        else:
            neither.append(month)
    point_gap[i] = (point, gap, neither)
    print(f'LSOA {i}')


dict_keys(['Barking and Dagenham 001A', 'Barking and Dagenham 001B', 'Barking and Dagenham 001C', 'Barking and Dagenham 001D', 'Barking and Dagenham 002A', 'Barking and Dagenham 002B', 'Barking and Dagenham 002C', 'Barking and Dagenham 002D', 'Barking and Dagenham 002E', 'Barking and Dagenham 002F', 'Barking and Dagenham 003A', 'Barking and Dagenham 003B', 'Barking and Dagenham 003C', 'Barking and Dagenham 003D', 'Barking and Dagenham 004A', 'Barking and Dagenham 004B', 'Barking and Dagenham 004C', 'Barking and Dagenham 004D', 'Barking and Dagenham 004E', 'Barking and Dagenham 004F', 'Barking and Dagenham 004G', 'Barking and Dagenham 006A', 'Barking and Dagenham 006B', 'Barking and Dagenham 006C', 'Barking and Dagenham 006D', 'Barking and Dagenham 006E', 'Barking and Dagenham 007A', 'Barking and Dagenham 007B', 'Barking and Dagenham 007C', 'Barking and Dagenham 007D', 'Barking and Dagenham 007E', 'Barking and Dagenham 007F', 'Barking and Dagenham 007G', 'Barking and Dagenham 008A', 'Ba

In [8]:
durations = {}
df = burglar_df['Month'].unique()
len_ = len(point_gap.keys())
count = 0
prevp = 0
for i in point_gap.keys():
    print(i)
    distance = []
    point = point_gap[i][0]
    gap = point_gap[i][1]
    neither = point_gap[i][2]
    diffs = []
    for month in df:
        if month in point:
            to_gap = 0
            max_val = 1
            trend = False
            dt_month = datetime.strptime(month, '%Y-%m')
            gap_c = [datetime.strptime(date, '%Y-%m') for date in gap]
            neither_c = [datetime.strptime(date, '%Y-%m') for date in neither]
            for j in range(12):
                current_date = dt_month + timedelta(days=30*j)  # adding approximately a month each time

                # Check if the current date is in the gap list
                if current_date in gap_c:
                    current_string = current_date.strftime('%Y-%m')
                    print(f'Point at {month}, Gap at {current_string}. Difference {diffs}')
                    distance.append((dt_month, to_gap, trend, tuple(diffs)))
                    break
                else:
                    to_gap += 1
                    if current_date in neither_c and LSOA_per_month[i][current_date.strftime('%Y-%m')] > max_val:
                      diffs.append(LSOA_per_month[i][current_date.strftime('%Y-%m')] - max_val)
                      max_val = LSOA_per_month[i][current_date.strftime('%Y-%m')]
                      trend = True
    durations[i] = distance



Streaming output truncated to the last 5000 lines.
Point at 2018-01, Gap at 2018-05. Difference [2, 1, 1, 3]
Point at 2018-02, Gap at 2018-07. Difference [2, 1, 1, 3, 1]
Point at 2018-04, Gap at 2018-05. Difference [2, 1, 1, 3, 1]
Point at 2018-11, Gap at 2019-03. Difference [2, 1, 1, 3, 1, 3]
Point at 2019-02, Gap at 2019-06. Difference [2, 1, 1, 3, 1, 3]
Point at 2021-02, Gap at 2021-06. Difference [2, 1, 1, 3, 1, 3]
Point at 2022-11, Gap at 2022-12. Difference [2, 1, 1, 3, 1, 3, 1, 1]
Point at 2023-01, Gap at 2023-04. Difference [2, 1, 1, 3, 1, 3, 1, 1]
Point at 2023-06, Gap at 2023-07. Difference [2, 1, 1, 3, 1, 3, 1, 1]
Point at 2024-09, Gap at 2024-10. Difference [2, 1, 1, 3, 1, 3, 1, 1, 3]
Point at 2024-11, Gap at 2024-12. Difference [2, 1, 1, 3, 1, 3, 1, 1, 3]
Waltham Forest 016F
Point at 2011-01, Gap at 2011-04. Difference []
Point at 2012-11, Gap at 2013-03. Difference [5]
Point at 2014-02, Gap at 2014-07. Difference [5, 2, 2, 2]
Point at 2014-06, Gap at 2014-07. Difference [

In [9]:
#6 Calculate mean and stdev distances for each LSOA
dur_mean = {}
for i in LSOA_per_month.keys():
  if i not in durations.keys():
    dur_mean[i] = []
    continue
  dur_val = durations[i]
  dur_mean[i] = [j[1] for j in dur_val]
mean_durations = {}
for i in dur_mean.keys():
    mean_durations[i] = [(float(np.mean(dur_mean[i])), float(np.std(dur_mean[i]))) if len(dur_mean[i]) != 0 else 0]
print(mean_durations)
assert len(mean_durations) == len(LSOA_per_month)

#7 For each LSOA, calculate the following:
#1 Likelihood of an upwards trend, the 'recency problem';
#2 Mean and stdev of all upwards trends

{'Barking and Dagenham 001A': [(2.0, 1.4142135623730951)], 'Barking and Dagenham 001B': [(2.0, 1.1726039399558574)], 'Barking and Dagenham 001C': [(1.9230769230769231, 1.327898192433236)], 'Barking and Dagenham 001D': [(1.9473684210526316, 1.3562209186026448)], 'Barking and Dagenham 002A': [(1.5, 1.0)], 'Barking and Dagenham 002B': [(2.888888888888889, 1.5947444549341472)], 'Barking and Dagenham 002C': [(2.2857142857142856, 1.097306535409801)], 'Barking and Dagenham 002D': [(1.6666666666666667, 0.9428090415820634)], 'Barking and Dagenham 002E': [(2.4166666666666665, 1.4976833963009524)], 'Barking and Dagenham 002F': [(2.1666666666666665, 1.674979270186815)], 'Barking and Dagenham 003A': [(2.5, 1.284523257866513)], 'Barking and Dagenham 003B': [(2.0952380952380953, 1.4443572274383905)], 'Barking and Dagenham 003C': [(2.1052631578947367, 1.2934953397310007)], 'Barking and Dagenham 003D': [(2.533333333333333, 1.3597385369580757)], 'Barking and Dagenham 004A': [(2.1875, 1.3792547806696194)

In [10]:
#7.0 Loading the data
print(durations.keys())
dur_trend = {}
dur_up = {}
for i in durations.keys():
  dur_val = durations[i]
  dur_trend[i] = [j[2] for j in dur_val]
  dur_up[i] = [list(j[3]) for j in dur_val]
print(dur_up)


#7.1
trend_likelihood = {}
for i in dur_trend.keys():
  trend_likelihood[i] = (len([j for j in dur_trend[i] if j]) / len(dur_trend))
print(trend_likelihood)
for i in trend_likelihood.keys():
  print(f'LSOA {i}: {trend_likelihood[i]}')

#7.2
trends_total = {}
for i in dur_up.keys():
  trends = []
  for j in dur_up[i]:
    trends.extend(j)
  trends_total[i] = trends
print(trends_total)
mean_trends = {}
for i in trends_total.keys():
  mean_trends[i] = [(float(np.mean(trends_total[i])), float(np.std(trends_total[i]))) if len(trends_total[i]) != 0 else 0]
for i in mean_trends.keys():
  print(f'LSOA {i}: {mean_trends[i]}')

Streaming output truncated to the last 5000 lines.
LSOA Barnet 027E: [(1.8333333333333333, 0.9860132971832694)]
LSOA Barnet 027F: [(1.9120879120879122, 0.7360176725116637)]
LSOA Barnet 028A: [(1.9655172413793103, 1.325681571499385)]
LSOA Barnet 028B: [(1.5483870967741935, 0.9448915171533424)]
LSOA Barnet 028C: [(1.7916666666666667, 1.189508535301683)]
LSOA Barnet 028D: [(1.816326530612245, 1.0368739528303483)]
LSOA Barnet 028E: [(1.5913978494623655, 0.9185842208246684)]
LSOA Barnet 029A: [(2.048780487804878, 1.228745533444878)]
LSOA Barnet 029B: [(1.8, 0.8573214099741124)]
LSOA Barnet 029C: [(1.8636363636363635, 0.9315409787235999)]
LSOA Barnet 029D: [(1.2647058823529411, 0.4411764705882353)]
LSOA Barnet 029E: [(1.4852941176470589, 0.4997836902343067)]
LSOA Barnet 029F: [(1.6666666666666667, 0.9428090415820634)]
LSOA Barnet 030A: [(1.8737864077669903, 0.9920030892772659)]
LSOA Barnet 030B: [(1.9230769230769231, 0.2664693550105965)]
LSOA Barnet 030D: [(2.1020408163265305, 1.182265172078

In [11]:
#8 Save All Findings
total = (mean_durations, trend_likelihood, mean_trends)
with open('total.txt', 'w') as f:
    f.write(str(total))
#